In [44]:
import pandas as pd
import numpy as np
import datetime as dt 
from collections import namedtuple

In [ ]:
gas_data = pd.read_csv("clean_df.csv")
len(gas_data)

In [ ]:
#preprocessing data for easier computation in the following 
start_time = timeit.default_timer()
gas_data.localminute = gas_data.localminute.str.slice(0,19)
gas_data.localminute = pd.to_datetime(gas_data.localminute, infer_datetime_format = True, 
                                      format = "%Y/%m/%d %I:%M:%S %p");
gas_data.localminute = gas_data.localminute.map(lambda x:x.replace(minute=0, second=0));
gas_data['meter_value']=gas_data['meter_value'].astype(float)

In [ ]:
#gas_data=gas_data[gas_data['dataid']==35];
ind=0;
_hr=dt.timedelta(hours=1); #creating a constant timedelta object with value =1 hr for comparison
temp_gas_hr=pd.DataFrame(columns=gas_data.columns);
temp_gas_hr=gas_data;
id_list=gas_data['dataid'].unique();#creating an id list for itertaion since only comparison of reading within id 
                                     #is meanningful

missing_lm=[];
missing_id=[];
missing_val=[];

for _id in id_list:
    #print(ind);
    #ind=ind+1;
    temp_gas_data=gas_data[gas_data['dataid']==_id];
    temp_gas_data.reset_index(drop=True,inplace=True);
    for  row in temp_gas_data.itertuples():
        if(row.Index==0):
            prev_row=pd.Series(data=[row.localminute,row.dataid,row.meter_value]
                               ,index=['localminute','dataid','meter_value']);
        else:
            time_diff=row.localminute-prev_row.localminute;
            if(time_diff>_hr):
                time_diff=int(time_diff.total_seconds()/3600);
                for j in range (1,time_diff):
                    if ((row.meter_value-prev_row.meter_value)>4):
                        acc_reading=float((row.meter_value-prev_row.meter_value)/time_diff);
                    else:
                        acc_reading=0;
                    time_change=dt.timedelta(hours=j);
                    new_time=prev_row.localminute+time_change;
                    missing_lm.append(new_time);
                    missing_id.append(_id);
                    missing_val.append(float(prev_row.meter_value+acc_reading*j));

        prev_row=pd.Series(data=[row.localminute,row.dataid,row.meter_value]
                               ,index=['localminute','dataid','meter_value']);
       


In [ ]:
missing_data={'localminute':missing_lm,'dataid':missing_id,'meter_value':missing_val};
missing_data=pd.DataFrame(missing_data,columns=gas_data.columns);
temp_gas_hr=pd.concat([gas_data,missing_data])

In [ ]:
temp_gas_hr.drop_duplicates(['localminute','dataid'],keep='last',inplace=True);
temp_gas_hr=temp_gas_hr.sort_values(by=['dataid','localminute']);
temp_gas_hr['meter_value']=temp_gas_hr['meter_value'].astype(int)
print("total: %ds" %(timeit.default_timer() - start_time))

In [ ]:
temp_gas_hr.to_csv('hourly_readings_final.csv',index=False);

In [ ]:
gas_hr=temp_gas_hr;
gas_hr=gas_hr[(gas_hr['localminute'].dt.year==2015)&(gas_hr['localminute'].dt.month==11)]
for _id in id_list:
    temp_id_hr=gas_hr[gas_hr['dataid']==_id];
    t='Hourly reading of '+str(_id);
    fig=plt.figure(figsize=(15,15))
    plt.plot(temp_id_hr.localminute,temp_id_hr.meter_value);
    plt.xlabel('date hr/(hr)');
    plt.ylabel('meter_value'),
    plt.title(t,loc='center');
    plt.grid();
    t_fig=t+'.png'
    fig.savefig(t_fig)
    plt.show();

In [45]:
hourly_reading = pd.read_csv('./csv/hourly_readings_final.csv')
dataid = hourly_reading['dataid'][0]
prev_reading= hourly_reading['meter_value'][0]

# compute hourly consumption by each household by finding the difference between adjacent readings
print(dataid)
for row in hourly_reading.itertuples():
    if getattr(row, 'dataid') == dataid:
        hourly_reading['meter_value'][getattr(row, 'Index')] -= prev_reading
    else:
        dataid = getattr(row, 'dataid')
        hourly_reading['meter_value'][getattr(row, 'Index')] = 0
        print(dataid)
    prev_reading = getattr(row, 'meter_value')
    
# print out the new dataframe to check values
print(hourly_reading)

# sort values and export as csv file
hourly_reading = hourly_reading.sort_values(by = ['dataid', 'localminute'])
hourly_reading.to_csv('hourly_consumption.csv', index = False)

35


F:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
F:\Anaconda\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


44
77
94
114
187
222
252
370
483
484
661
739
744
871
1042
1086
1103
1185
1283
1403
1415
1507
1556
1589
1619
1697
1714
1718
1790
1791
1792
1800
1801
2018
2034
2072
2094
2129
2233
2335
2378
2449
2461
2470
2575
2638
2645
2755
2814
2818
2945
2946
2965
2980
3036
3039
3134
3310
3367
3527
3544
3577
3635
3723
3778
3849
3893
3918
4029
4031
4193
4228
4296
4352
4356
4373
4421
4447
4514
4671
4732
4767
4874
4998
5129
5131
5193
5275
5317
5395
5403
5439
5484
5545
5636
5658
5785
5810
5814
5892
5972
6101
6412
6505
6578
6673
6685
6830
6836
6863
6910
7016
7017
7030
7117
7287
7429
7460
7566
7674
7682
7739
7741
7794
7900
7919
7965
7989
8059
8084
8086
8155
8156
8244
8386
8467
8703
8829
8890
8967
9052
9121
9134
9160
9278
9295
9474
9600
9620
9631
9639
9729
9766
9849
9956
9982
                localminute  dataid  meter_value
0       2015-10-01 00:00:00      35            0
1       2015-10-01 01:00:00      35            0
2       2015-10-01 02:00:00      35            0
3       2015-10-01 03:00:00      35      

In [46]:
# 1.3
hourly_reading = pd.read_csv('./csv/hourly_readings_final.csv')

# get the list of dataids
idList = hourly_reading['dataid'].drop_duplicates(keep = 'first')
idList = idList.reset_index(drop = True)

# create a dataframe with only meter_values for each household
df = pd.DataFrame(columns = idList)
for i in range(0, len(idList)):
    df[idList[i]] = hourly_reading.loc[hourly_reading.dataid == idList[i]]['meter_value'].reset_index(drop = True)
corr = df.corr()

# remove the 1s in diagonal
corr -= np.eye(corr.shape[0])

# create a new dataframe with random values inserted
top_corr = pd.DataFrame(np.random.randint(low = 0.0, high = 10.0, size = (5 * len(idList), 3)), columns = ['HH1', 'HH2', 'corr']).reset_index(drop = True)
# set 'corr' column data type as float64
top_corr['corr'] = top_corr['corr'].astype(float)

# for each column, find the 5 values with the highest correlation values and insert into top_corr dataframe
count = 0
for i in corr.columns[:]:
    temp_corr = corr.nlargest(5, i)
    for j in range (0, 5):
        top_corr['HH1'][count] = i
        top_corr['HH2'][count] = temp_corr.index[j]
        top_corr['corr'][count] = temp_corr[i].iloc[j]
        count += 1

# export top_corr dataframe as csv file
top_corr.to_csv('top_correlated_households.csv', index = False)


F:\Anaconda\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
F:\Anaconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
F:\Anaconda\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
